In [1]:
require 'stopwords'
require 'rblearn'
require 'stemmify'
require 'numo/narray'
require 'rblearn'

false

# 70. データの入手・整形

文に関する極性分析の正解データを用い，以下の要領で正解データ（sentiment.txt）を作成せよ．

rt-polarity.posの各行の先頭に"+1 "という文字列を追加する（極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
rt-polarity.negの各行の先頭に"-1 "という文字列を追加する（極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
上述1と2の内容を結合（concatenate）し，行をランダムに並び替える
sentiment.txtを作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．

In [2]:
def knock70
  pos = File.open("../data/rt-polaritydata/rt-polarity.pos").read
  neg = File.open("../data/rt-polaritydata/rt-polarity.pos").read

  pos_tagged = pos.split("\n").map {|line| "+1 #{line}"}
  neg_tagged = neg.split("\n").map {|line| "-1 #{line}"}
  sen = (pos_tagged + neg_tagged).shuffle
  counter = {"+1" => 0, "-1" => 0}

  out = File.open("../data/sentiment.txt", "w")
  sen.each do |line|
    out.puts line
  end
  out.close

  open('../data/sentiment.txt').each_line do |line|
    counter[line[0..1]] += 1
  end

  return counter
end

:knock70

In [3]:
knock70

{"+1"=>5330, "-1"=>5330}

# 71. ストップワード
英語のストップワードのリスト（ストップリスト）を適当に作成せよ．さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，それ以外は偽を返す関数を実装せよ．さらに，その関数に対するテストを記述せよ．

In [4]:
# in order to check if the word is stop word with O(1)
Stop_words = Hash.new {|hash, key| hash[key] = false}
(Stopwords::STOP_WORDS + ["'", ",", '.', ":", ";"]).each {|stop_word| Stop_words[stop_word] = true}

def is_stop?(word)
  Stop_words[word]
end

:is_stop?

# 72. 素性抽出

極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．

In [5]:
def knock72
  labels = []
  features = []

  File.foreach('../data/sentiment.txt') do |line|
    line.chomp!
    label = line[0..1]
    feature = line[3..-1]
    labels << (label == '+1' ? 1 : 0)
    features << feature
  end

  cv = Rblearn::CountVectorizer.new(lambda{|feature| feature.split.map(&:stem).map(&:downcase)}, 1, 0.7)
  cv.fit_transform(features)

  y = Numo::Int8.zeros([labels.size, 1])
  labels.each_with_index do |label, i|
    y[i, 0] = label
  end

  return [cv, y]
end

:knock72

In [6]:
cv, y = knock72

1

1

# 73. 学習

72で抽出した素性を用いて，ロジスティック回帰モデルを学習せよ．

In [7]:
class LogisticRegressionModel
  attr_accessor :w

  def initialize(eta, threshold)
    @eta = eta
    @threshold = threshold
  end
  
  
  def h(x)
    z = x.dot(@w)
    return 1 / (1 + Numo::NMath.exp(-z))
  end


  def fit(x, y)
    @w = Numo::Float64.new([x.shape[1], 1]).rand
    30.times do |t|

      # p y-h(x)
      # NOTE: there is something bug
      @w = @w - @eta * x.transpose.dup.dot(h(x) - y) # x.T.dot(h(x) - y): gradient
      puts "#iter: #{t}, accuracy: #{score(x, y)}" if t % 5 == 0
    end
  end


  def predict(x)
    probabilities = h(x)
    return probabilities > @threshold
  end


  def predict_prob(x)
    return h(x)
  end


  def score(x, y)
    predicted_class = predict(x)
    num_data = predicted_class.shape[0]

    py = predicted_class.to_a
    ry = y.to_a

    num_correct = 0
    num_data.times do |t|
      num_correct += 1 if py[t] == ry[t]
    end
    return num_correct.to_f / num_data
  end
end

:score

In [8]:
x = cv.doc_matrix
x_train, y_train, x_test, y_test = Rblearn::CrossValidation.train_test_split(x, y, 0.5).map(&:dup)

model = LogisticRegressionModel.new(0.02, 0.7)
model.fit(x_train, y_train)

puts "test accuracy: #{model.score(x_test, y_test)}"

#iter: 0, accuracy: 0.5069418386491558
#iter: 5, accuracy: 0.5560975609756098
#iter: 10, accuracy: 0.5887429643527204
#iter: 15, accuracy: 0.6080675422138837
#iter: 20, accuracy: 0.6213883677298312
#iter: 25, accuracy: 0.6311444652908067
test accuracy: 0.3617260787992495


# 74. 予測

73で学習したロジスティック回帰モデルを用い，与えられた文の極性ラベル（正例なら"+1"，負例なら"-1"）と，その予測確率を計算するプログラムを実装せよ．

In [9]:
model.predict(x_test).each_with_index do |arr, index|
  puts arr
  break if index == 30
end

0
1
1
1
1
0
0
0
0
0
0
1
1
1
0
1
0
0
0
0
1
0
0
1
1
0
0
0
0
0
0


# 75. 素性の重み
73で学習したロジスティック回帰モデルの中で，重みの高い素性トップ10と，重みの低い素性トップ10を確認せよ．

In [10]:
feature_names = cv.get_feature_names
weights = model.w.to_a.map{|arr| arr[0]}
name_weights = {}

feature_names.size.times do |t|
  name_weights[feature_names[t]] = weights[t]
end

puts "high"
name_weights.
  sort{|(_, v1), (_, v2)| v2 <=> v1}.
  first(10).
  map{|arr| arr.join("\t")}.
  each do |record|
    puts record
  end

puts "low"
name_weights.
  sort{|(_, v1), (_, v2)| v1 <=> v2}.
  first(10).
  map{|arr| arr.join("\t")}.
  each do |record|
    puts record
  end

1

high
spark	1.597826925085305
progress	1.4879646682207617
dead	1.444739489440449
flair	1.4342877810955545
stuart	1.4147933426740569
sear	1.4098002203917002
gloss	1.4086219310397996
muccino	1.4040320651507008
nasti	1.341434823216564
nostalgia	1.33791615251975
low
y	-1.3076134455471835
tradit	-1.2918763845310999
cinemat	-1.2580662063052148
john	-1.2508738545571203
roll	-1.153649362359493
strike	-1.1415180912167298
simpli	-1.1066214824500504
antic	-1.1049492821687918
talent	-1.0399403104165472
center	-1.0210059418976267


1

# 76. ラベル付け

学習データに対してロジスティック回帰モデルを適用し，正解のラベル，予測されたラベル，予測確率をタブ区切り形式で出力せよ．

In [11]:
a = y_test.to_a.map{|arr| arr[0]}
b = model.predict(x_test).to_a.map{|arr| arr[0]}
c = model.predict_prob(x_test).to_a.map{|arr| arr[0]}

output = open('../data/knock76.tsv', 'w')

a.size.times do |t|
  puts "#{a[t]}\t#{b[t]}\t#{c[t]}" if t < 30
  output.puts "#{a[t]}\t#{b[t]}\t#{c[t]}"
end

output.close

1	0	0.1314689978032677
0	1	0.7461735800854119
1	1	0.837494327018176
0	1	0.7643645161495609
1	1	0.7581279822014793
0	0	0.5109588535089932
0	0	0.6795177559043514
1	0	0.6415005470906667
1	0	0.10308033650702865
0	0	0.4242130718630955
0	0	0.22791969699227949
0	1	0.8655672849209247
0	1	0.7600173254693662
1	1	0.7248224262153257
0	0	0.49960708329416625
0	1	0.8207661204919647
1	0	0.3264702427186831
1	0	0.45934594377098
1	0	0.5360512053477997
0	0	0.04588150035881643
0	1	0.9766663346991744
1	0	0.339681458259064
0	0	0.33295423204546004
0	1	0.9110690191626709
1	1	0.8829097226455447
1	0	0.3819298246100228
0	0	0.4469917806206784
1	0	0.42731635036797644
1	0	0.20725726477480755
1	0	0.5177972063551768


# 77. 正解率の計測

76の出力を受け取り，予測の正解率，正例に関する適合率，再現率，F1スコアを求めるプログラムを作成せよ．

In [12]:
num_all = 0
num1 = 0.0
num2 = 0.0
num3 = 0.0

# a is the actual label
# b is the predicted label
File.foreach('../data/knock76.tsv') do |line|
  num_all += 1

  a, b, _ = line.split.map(&:to_f)
  
  if b == 1
    if a == 1
      num1 += 1
    elsif a == 0
      num2 += 1
    end
  elsif b == 0 && a == 1
    num3 += 1
  end
end

precision = num1 / (num1 + num2)
recall = num3 / (num1 + num3)

puts "precision: #{precision}"
puts "recall: #{recall}"
puts "F-1 score: #{2*precision*recall / (precision + recall)}"

precision: 0.2686002522068096
recall: 0.8403298350824587
F-1 score: 0.4070821203738828
